In [1]:
!pip install -q langchain pypdf sentence-transformers chromadb openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.0/237.0 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
# 라이브러리
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

In [3]:
# 웹페이지 불러오기
loader = WebBaseLoader("https://www.momentoushistory.com/byeongjahoran-second-manchu-invasion-korea-1636/")
data = loader.load()

In [4]:
# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=0)
splits = text_splitter.split_documents(data)

In [5]:
# 벡터DB
model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {'normalize_embeddings': True}
ko_embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

vectordb = Chroma.from_documents(documents=splits, embedding=ko_embedding)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chat_models import ChatOpenAI

question = "병자호란의 역사적 배경이 뭐야?"
llm = ChatOpenAI(temperature=0, openai_api_key = "...")
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm
)

In [10]:
# 쿼리 로깅
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [11]:
unique_docs = retriever_from_llm.get_relevant_documents(query=question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. 병자호란의 역사적 배경은 무엇인가요?', '2. 병자호란이 일어난 배경에 대해 알려주세요.', '3. 병자호란이 발생한 역사적인 배경은 어떻게 되나요?']


6

In [12]:
unique_docs

[Document(page_content='1. 병자호란의 시작과 그 역사적 배경\n병자호란은 한국 역사에서 중요한 전환점을 마련한 사건입니다. 1636년부터 1637년에 걸쳐 일어난 이 전쟁은, 조선과 청나라 간의 긴장이 고조된 결과였습니다. 그 배경을 살펴보면, 청나라의 급속한 팽창과 명나라의 쇠약, 그리고 조선의 대외 정책이 주요 원인으로 작용했습니다.', metadata={'language': 'ko-KR', 'source': 'https://www.momentoushistory.com/byeongjahoran-second-manchu-invasion-korea-1636/', 'title': '병자호란: 조선과 청의 격동의 시기 - 모멘터스 히스토리'}),
 Document(page_content='병자호란은 조선뿐만 아니라 동아시아 역사의 흐름에 중대한 영향을 끼친 사건으로, 그 여파는 오랜 시간 동안 이어졌습니다. 이 전쟁은 조선의 대외 정책, 사회 구조, 그리고 국민 정신에 큰 변화를 가져왔고, 이는 후세에 이르기까지 한국 역사에 중요한 영향을 미쳤습니다.', metadata={'language': 'ko-KR', 'source': 'https://www.momentoushistory.com/byeongjahoran-second-manchu-invasion-korea-1636/', 'title': '병자호란: 조선과 청의 격동의 시기 - 모멘터스 히스토리'}),
 Document(page_content='2. 병자호란의 전개: 전쟁의 진행 과정\n병자호란의 발발은 조선에 큰 충격을 주었습니다. 청나라의 군대는 조선의 북방 경계를 빠르게 돌파하며 한양으로 진격했습니다. 조선의 군대는 청의 강력한 군사력에 맞서 싸웠지만, 결국 여러 차례의 패배를 겪었습니다. 이 과정에서 조선의 군사적, 정치적 리더십의 한계가 드러났고, 이는 조선 내부에도 큰 영향을 미쳤습니다.', metadata={'language': 'ko-KR', 'source': 'https: